## 추가 실험

> #6-1 Age Initial로 예측 valid 0.866 lb 0.748

> #6-2 Age Initial로 예측 + Initial 컬럼 학습에서 제외 valid 0.866 lb 0.741

> #7 SibSp, Parch : family, solo 파생변수 추가 valid 0.819 lb 0.755

> #8 오버샘플링 valid 0.824 lb 0.737

In [2]:
from catboost import CatBoostClassifier, CatBoostRegressor
import catboost as cb

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

In [40]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submit = pd.read_csv('submission.csv')

train['Embarked'] = train['Embarked'].fillna('S')
test.loc[test[test.Fare.isnull()].index, 'Fare'] = 8.0500

train['Sex'] = train['Sex'].replace('male', 0)
train['Sex'] = train['Sex'].replace('female', 1)
train['Embarked'] = train['Embarked'].replace('C', 0)
train['Embarked'] = train['Embarked'].replace('Q', 1)
train['Embarked'] = train['Embarked'].replace('S', 2)
train.drop(columns=['PassengerId', 'Name'], inplace=True)

test['Sex'] = test['Sex'].replace('male',0)
test['Sex'] = test['Sex'].replace('female',1)
test['Embarked'] = test['Embarked'].replace('C', 0)
test['Embarked'] = test['Embarked'].replace('Q', 1)
test['Embarked'] = test['Embarked'].replace('S', 2)
test.drop(columns=['PassengerId', 'Name'], inplace=True)

In [21]:
_train = pd.read_csv('train.csv')
_test = pd.read_csv('test.csv')

_train['Sex'] = _train['Sex'].replace('male', 0)
_train['Sex'] = _train['Sex'].replace('female', 1)
_train['Embarked'] = _train['Embarked'].replace('C', 0)
_train['Embarked'] = _train['Embarked'].replace('Q', 1)
_train['Embarked'] = _train['Embarked'].replace('S', 2)
_train.drop(columns=['PassengerId'], inplace=True)

_test['Sex'] = _test['Sex'].replace('male',0)
_test['Sex'] = _test['Sex'].replace('female',1)
_test['Embarked'] = _test['Embarked'].replace('C', 0)
_test['Embarked'] = _test['Embarked'].replace('Q', 1)
_test['Embarked'] = _test['Embarked'].replace('S', 2)
_test.drop(columns=['PassengerId'], inplace=True)

_test.loc[_test[_test.Fare.isnull()].index, 'Fare'] = 8.0500

_train['Initial'] = 0
for i in _train:
    _train['Initial'] = _train['Name'].str.extract('([A-Za-z]+)\.') # .을 기준으로 앞에 있는 부분을 추출하는 정규표현식
    
_test['Initial']=0
for i in _test:
    _test['Initial'] = _test.Name.str.extract('([A-Za-z]+)\.')
    
_train['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],['Miss','Miss','Miss','Mr','Mrs','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'], inplace = True)
_test['Initial'].replace(['Rev','Col','Dr','Dona','Ms'],['Other','Other','Mr','Mr',"Miss"],inplace=True)    

_train.loc[(_train.Age.isnull()) & (_train.Initial=='Mr'),'Age'] = 33
_train.loc[(_train.Age.isnull()) & (_train.Initial=='Mrs'),'Age'] = 36
_train.loc[(_train.Age.isnull()) & (_train.Initial=='Master'),'Age'] = 5
_train.loc[(_train.Age.isnull()) & (_train.Initial=='Miss'),'Age'] = 22
_train.loc[(_train.Age.isnull()) & (_train.Initial=='Other'),'Age'] = 46

_test.loc[(_test.Age.isnull()) & (_test.Initial=='Mr'),'Age'] = 33
_test.loc[(_test.Age.isnull()) & (_test.Initial=='Mrs'),'Age'] = 36
_test.loc[(_test.Age.isnull()) & (_test.Initial=='Master'),'Age'] = 5
_test.loc[(_test.Age.isnull()) & (_test.Initial=='Miss'),'Age'] = 22
_test.loc[(_test.Age.isnull()) & (_test.Initial=='Other'),'Age'] = 46

_train['Initial'] = _train['Initial'].astype('category').cat.codes
_test['Initial'] = _test['Initial'].astype('category').cat.codes

_train, valid = train_test_split(_train, test_size = 0.1, random_state=42)
train_x, train_y = _train.drop(columns=['Survived']), _train['Survived']
valid_x, valid_y = valid.drop(columns=['Survived']), valid['Survived']

for data in [train_x, valid_x, _test]:
    data.drop(columns=['Cabin', 'Ticket', 'Name'], inplace=True)

train_dataset = cb.Pool(train_x, train_y) 
valid_dataset = cb.Pool(valid_x, valid_y)
test_dataset = cb.Pool(_test)

model = cb.CatBoostClassifier(loss_function='Logloss', silent=True, random_state=42)
model.fit(train_dataset)

pred = model.predict(valid_dataset)
score = roc_auc_score(valid_y.values, pred)
print(score)

submit['Survived'] = model.predict(test_dataset)
submit.to_csv('#6-1.csv', index=False)

0.8657407407407408


In [22]:
_train = pd.read_csv('train.csv')
_test = pd.read_csv('test.csv')

_train['Sex'] = _train['Sex'].replace('male', 0)
_train['Sex'] = _train['Sex'].replace('female', 1)
_train['Embarked'] = _train['Embarked'].replace('C', 0)
_train['Embarked'] = _train['Embarked'].replace('Q', 1)
_train['Embarked'] = _train['Embarked'].replace('S', 2)
_train.drop(columns=['PassengerId'], inplace=True)

_test['Sex'] = _test['Sex'].replace('male',0)
_test['Sex'] = _test['Sex'].replace('female',1)
_test['Embarked'] = _test['Embarked'].replace('C', 0)
_test['Embarked'] = _test['Embarked'].replace('Q', 1)
_test['Embarked'] = _test['Embarked'].replace('S', 2)
_test.drop(columns=['PassengerId'], inplace=True)

_test.loc[_test[_test.Fare.isnull()].index, 'Fare'] = 8.0500

_train['Initial'] = 0
for i in _train:
    _train['Initial'] = _train['Name'].str.extract('([A-Za-z]+)\.') # .을 기준으로 앞에 있는 부분을 추출하는 정규표현식
    
_test['Initial']=0
for i in _test:
    _test['Initial'] = _test.Name.str.extract('([A-Za-z]+)\.')
    
_train['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],['Miss','Miss','Miss','Mr','Mrs','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'], inplace = True)
_test['Initial'].replace(['Rev','Col','Dr','Dona','Ms'],['Other','Other','Mr','Mr',"Miss"],inplace=True)    

_train.loc[(_train.Age.isnull()) & (_train.Initial=='Mr'),'Age'] = 33
_train.loc[(_train.Age.isnull()) & (_train.Initial=='Mrs'),'Age'] = 36
_train.loc[(_train.Age.isnull()) & (_train.Initial=='Master'),'Age'] = 5
_train.loc[(_train.Age.isnull()) & (_train.Initial=='Miss'),'Age'] = 22
_train.loc[(_train.Age.isnull()) & (_train.Initial=='Other'),'Age'] = 46

_test.loc[(_test.Age.isnull()) & (_test.Initial=='Mr'),'Age'] = 33
_test.loc[(_test.Age.isnull()) & (_test.Initial=='Mrs'),'Age'] = 36
_test.loc[(_test.Age.isnull()) & (_test.Initial=='Master'),'Age'] = 5
_test.loc[(_test.Age.isnull()) & (_test.Initial=='Miss'),'Age'] = 22
_test.loc[(_test.Age.isnull()) & (_test.Initial=='Other'),'Age'] = 46

_train, valid = train_test_split(_train, test_size = 0.1, random_state=42)
train_x, train_y = _train.drop(columns=['Survived']), _train['Survived']
valid_x, valid_y = valid.drop(columns=['Survived']), valid['Survived']

for data in [train_x, valid_x, _test]:
    data.drop(columns=['Cabin', 'Ticket', 'Name', 'Initial'], inplace=True)

train_dataset = cb.Pool(train_x, train_y) 
valid_dataset = cb.Pool(valid_x, valid_y)
test_dataset = cb.Pool(_test)

model = cb.CatBoostClassifier(loss_function='Logloss', silent=True, random_state=42)
model.fit(train_dataset)

pred = model.predict(valid_dataset)
score = roc_auc_score(valid_y.values, pred)
print(score)

submit['Survived'] = model.predict(test_dataset)
submit.to_csv('#6-2.csv', index=False)

0.8657407407407408


In [30]:
_train = train.copy()
_test = test.copy()

# family size 변수 생성
_train['family_size'] = _train['SibSp'] + _train['Parch'] + 1
_test['family_size'] = _test['SibSp'] + _test['Parch'] + 1

# solo 변수 생성
_train['solo'] = (_train['family_size'] == 1)
_test['solo'] = (_test['family_size'] == 1) 

_train, valid = train_test_split(_train, test_size = 0.1, random_state=42)
train_x, train_y = _train.drop(columns=['Survived']), _train['Survived']
valid_x, valid_y = valid.drop(columns=['Survived']), valid['Survived']

for data in [train_x, valid_x, _test]:
    data.drop(columns=['Age', 'Cabin', 'Ticket'], inplace=True)

train_dataset = cb.Pool(train_x, train_y) 
valid_dataset = cb.Pool(valid_x, valid_y)
test_dataset = cb.Pool(_test)

model = cb.CatBoostClassifier(loss_function='Logloss', silent=True, random_state=42)
model.fit(train_dataset)

pred = model.predict(valid_dataset)
score = roc_auc_score(valid_y.values, pred)
print(score)

submit['Survived'] = model.predict(test_dataset)
submit.to_csv('#7.csv', index=False)

0.8194444444444445


In [42]:
from imblearn.over_sampling import SMOTE

_train = train.copy()
_test = test.copy()

_train, valid = train_test_split(_train, test_size = 0.1, random_state=42)
train_x, train_y = _train.drop(columns=['Survived']), _train['Survived']
valid_x, valid_y = valid.drop(columns=['Survived']), valid['Survived']

for data in [train_x, valid_x, _test]:
    data.drop(columns=['Age', 'Cabin', 'Ticket'], inplace=True)

smote = SMOTE()
train_x_over, train_y_over = smote.fit_resample(train_x, train_y)
    
train_dataset = cb.Pool(train_x_over, train_y_over) 
valid_dataset = cb.Pool(valid_x, valid_y)
test_dataset = cb.Pool(_test)

model = cb.CatBoostClassifier(loss_function='Logloss', silent=True, random_state=42)
model.fit(train_dataset)

pred = model.predict(valid_dataset)
score = roc_auc_score(valid_y.values, pred)
print(score)

submit['Survived'] = model.predict(test_dataset)
submit.to_csv('#8.csv', index=False)

0.8240740740740741


## 종합 

> #final valid 0.837 lb 0.753
> #ensemble lb 0.751

- Age 양자화로 보간
- Age 예측에는 사용하지 않지만, Initial 컬럼 추가
- Cabin 카테고리 분류로 예측
- SibSp, Parch는 Family와 Solo로 변환

In [45]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submit = pd.read_csv('submission.csv')

train['Embarked'] = train['Embarked'].fillna('S')
test.loc[test[test.Fare.isnull()].index, 'Fare'] = 8.0500

train['Sex'] = train['Sex'].replace('male', 0)
train['Sex'] = train['Sex'].replace('female', 1)
train['Embarked'] = train['Embarked'].replace('C', 0)
train['Embarked'] = train['Embarked'].replace('Q', 1)
train['Embarked'] = train['Embarked'].replace('S', 2)
train.drop(columns=['PassengerId'], inplace=True)

test['Sex'] = test['Sex'].replace('male',0)
test['Sex'] = test['Sex'].replace('female',1)
test['Embarked'] = test['Embarked'].replace('C', 0)
test['Embarked'] = test['Embarked'].replace('Q', 1)
test['Embarked'] = test['Embarked'].replace('S', 2)
test.drop(columns=['PassengerId'], inplace=True)

_train = train.copy()
_test = test.copy()

##################################################

_train['Cabin'] = train['Cabin'].astype('category').cat.codes
_test['Cabin'] = test['Cabin'].astype('category').cat.codes

cabin_train, cabin_blank = _train[_train.Cabin != -1], _train[_train.Cabin == -1]
cabin_dataset = cb.Pool(cabin_train.drop(columns=['Survived', 'Cabin', 'Age', 'Ticket', 'Name']), cabin_train['Cabin'])

model = cb.CatBoostClassifier(loss_function='MultiClass', silent=True, random_state=42)
model.fit(cabin_dataset)

pred = model.predict(cabin_blank.drop(columns=['Survived', 'Cabin', 'Age', 'Ticket', 'Name']))
_train.loc[cabin_blank.index]['Cabin'] = np.reshape(pred, (-1,))

_, cabin_test = _test[_test.Cabin != -1], _test[_test.Cabin == -1]
cabin_dataset = cb.Pool(cabin_test.drop(columns=['Cabin', 'Age', 'Ticket', 'Name']))

pred = model.predict(cabin_dataset)
_test.loc[cabin_test.index]['Cabin'] = np.reshape(pred, (-1,))

##################################################

cols, qnt = ['Pclass', 'SibSp'], 3
_train.Age = pd.cut(_train.Age, qnt, labels=range(qnt))
table = _train[cols + ['Age']].value_counts().sort_index()

for index, *value in _train.loc[_train.Age.isnull(), cols].reset_index().values:
    for i in range(len(cols)):
        if (table.index.get_level_values(i) == value[i]).any():
            continue
        while not (table.index.get_level_values(i) == value[i]).any():
            value[i] -= 1
        break

    value = tuple(value)
    age = table.loc[value].sort_values(ascending=False).index[0]
    _train.loc[index, 'Age'] = age
    
_train.Age = _train.Age.astype(int)

_test.Age = pd.cut(test.Age, qnt, labels=range(qnt))

for index, *value in _test.loc[_test.Age.isnull(), cols].reset_index().values:
    for i in range(len(cols)):
        if (table.index.get_level_values(i) == value[i]).any():
            continue
        while not (table.index.get_level_values(i) == value[i]).any():
            value[i] -= 1
        break

    value = tuple(value)
    age = table.loc[value].sort_values(ascending=False).index[0]
    _test.loc[index, 'Age'] = age

_test.Age = _test.Age.astype(int)

##################################################

# family size 변수 생성
_train['family_size'] = _train['SibSp'] + _train['Parch'] + 1
_test['family_size'] = _test['SibSp'] + _test['Parch'] + 1

# solo 변수 생성
_train['solo'] = (_train['family_size'] == 1)
_test['solo'] = (_test['family_size'] == 1) 

##################################################

_train['Initial'] = 0
for i in _train:
    _train['Initial'] = _train['Name'].str.extract('([A-Za-z]+)\.') # .을 기준으로 앞에 있는 부분을 추출하는 정규표현식
    
_test['Initial']=0
for i in _test:
    _test['Initial'] = _test.Name.str.extract('([A-Za-z]+)\.')
    
_train['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],['Miss','Miss','Miss','Mr','Mrs','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'], inplace = True)
_test['Initial'].replace(['Rev','Col','Dr','Dona','Ms'],['Other','Other','Mr','Mr',"Miss"],inplace=True) 

_train['Initial'] = _train['Initial'].astype('category').cat.codes
_test['Initial'] = _test['Initial'].astype('category').cat.codes

##################################################

_train, valid = train_test_split(_train, test_size = 0.1, random_state=42)

train_x, train_y = _train.drop(columns=['Survived']), _train['Survived']
valid_x, valid_y = valid.drop(columns=['Survived']), valid['Survived']

for data in [train_x, valid_x, _test]:
    data.drop(columns=['Ticket', 'Name'], inplace=True)

train_dataset = cb.Pool(train_x, train_y) 
valid_dataset = cb.Pool(valid_x, valid_y)
test_dataset = cb.Pool(_test)

model = cb.CatBoostClassifier(loss_function='Logloss', silent=True, random_state=42)
model.fit(train_dataset)

pred = model.predict(valid_dataset)
score = roc_auc_score(valid_y.values, pred)
print(score)

submit['Survived'] = model.predict(test_dataset)
submit.to_csv('#final.csv', index=False)

0.8379629629629629


In [60]:
from glob import glob

submit = pd.read_csv('submission.csv')
submit.Survived = 0

filelist = glob('#*.csv')
for filename in filelist:
    tmp = pd.read_csv(filename)
    submit.Survived = submit.Survived + tmp.Survived.values
    
submit.Survived = submit.Survived.apply(lambda x: int(x >= len(filelist) // 2))
submit.to_csv('@ensemble.csv', index=False)